In [12]:
import carla

def display_road_topology(world, carla_map, lifetime=120.0, step=2.0):
    """
    Display unique waypoint numbers for each topology waypoint and then draw the path between them with the same color.
    
    Args:
        world (carla.World)
        carla_map (carla.Map)
        lifetime (float): Seconds to keep drawings
        step (float): Distance (meters) between waypoints
    """
    topology = carla_map.get_topology()
    debug = world.debug
    waypoint_counter = 0
    seen_waypoints = set()

    print(f"Drawing {len(topology)} road segments with unique waypoint numbers...")

    # Step 1: Display unique waypoints
    for wp_start, wp_end in topology:
        # Only draw the waypoint number if we haven't drawn this waypoint before
        if wp_start not in seen_waypoints:
            label_start = f"{waypoint_counter}"
            debug.draw_string(
                location=wp_start.transform.location + carla.Location(z=1.5),
                text=label_start,
                color=carla.Color(255, 0, 0),
                life_time=lifetime
            )
            seen_waypoints.add(wp_start)
            waypoint_counter += 1

    # Step 2: Draw the path between the unique waypoints
    for wp_start, wp_end in topology:
        current_wp = wp_start

        # Draw full path from wp_start to wp_end with the same color
        while current_wp.transform.location.distance(wp_end.transform.location) > step:
            next_wps = current_wp.next(step)
            if not next_wps:
                break

            next_wp = next_wps[0]

            # Draw the line connecting the consecutive waypoints
            debug.draw_line(
                begin=current_wp.transform.location + carla.Location(z=0.5),
                end=next_wp.transform.location + carla.Location(z=0.5),
                thickness=0.2,
                color=carla.Color(0, 255, 0),  # White color for the path
                life_time=lifetime
            )

            current_wp = next_wp

    print(f"Done drawing unique waypoints and paths.")

# Assuming you have world and map objects
client = carla.Client('localhost', 2000)
world = client.get_world()
map = world.get_map()

# Option 2: Display topology with connections
display_road_topology(world, map)

Drawing 200 road segments with unique waypoint numbers...
Done drawing unique waypoints and paths.


In [2]:
import sys
import carla
from collections import defaultdict

sys.path.append('C:/CARLA_0.9.15/PythonAPI/carla')
from agents.navigation.local_planner import RoadOption

def find_longest_non_junction_paths(carla_map, top_n=20, step=1.0):
    """
    Find the longest paths without intersections, ensuring no path overlaps with longer paths.
    
    Args:
        carla_map (carla.Map): The CARLA map
        top_n (int): Number of longest paths to return
        step (float): Distance between waypoints when exploring
        
    Returns:
        list: Top N longest non-overlapping paths as tuples (start_wp, end_wp, distance, waypoint_list)
    """
    topology = carla_map.get_topology()
    
    # Step 1: Build a graph of road segments (non-junction to non-junction)
    # We need to find continuous segments between junctions
    print("Building road segment graph...")
    
    all_segments = []
    processed_starts = set()
    
    # Get all waypoints from topology and explore from each
    for wp_start, wp_end in topology:
        # Try to find the beginning of a non-junction segment
        # Start from wp_start and go backwards until we hit a junction or dead end
        start_candidates = [wp_start]
        
        # Also check previous waypoints
        current = wp_start
        for _ in range(100):  # Limit backwards search
            prev_wps = current.previous(step)
            if not prev_wps or prev_wps[0].is_junction:
                break
            current = prev_wps[0]
            start_candidates.append(current)
        
        # Now explore forward from each candidate
        for start_wp in start_candidates:
            if start_wp.is_junction:
                continue
            
            # Create unique ID for this starting point
            start_id = (start_wp.road_id, start_wp.lane_id, 
                       round(start_wp.transform.location.x, 1), 
                       round(start_wp.transform.location.y, 1))
            
            if start_id in processed_starts:
                continue
            
            processed_starts.add(start_id)
            
            # Explore forward
            current_wp = start_wp
            total_distance = 0.0
            path_waypoints = [start_wp]
            
            while True:
                next_wps = current_wp.next(step)
                
                if not next_wps:
                    break
                
                next_wp = next_wps[0]
                
                # Stop if we reach a junction
                if next_wp.is_junction:
                    break
                
                # Calculate distance
                distance_increment = current_wp.transform.location.distance(
                    next_wp.transform.location
                )
                total_distance += distance_increment
                path_waypoints.append(next_wp)
                current_wp = next_wp
                
                # Safety check
                if len(path_waypoints) > 10000:
                    print(f"Warning: Path too long, breaking at {total_distance}m")
                    break
            
            # Save segment if it has meaningful length
            if total_distance > step:
                end_wp = path_waypoints[-1]
                all_segments.append((start_wp, end_wp, total_distance, path_waypoints))
    
    print(f"Found {len(all_segments)} total road segments")
    
    # Step 2: Sort by distance (longest first)
    all_segments.sort(key=lambda x: x[2], reverse=True)
    
    # Step 3: Select non-overlapping segments using a spatial approach
    selected_paths = []
    used_locations = set()
    
    for start_wp, end_wp, distance, waypoint_list in all_segments:
        # Check if this path overlaps with any already selected path
        path_overlaps = False
        
        # Sample waypoints along the path to check for overlaps
        sample_indices = list(range(0, len(waypoint_list), max(1, len(waypoint_list) // 10)))
        if len(waypoint_list) - 1 not in sample_indices:
            sample_indices.append(len(waypoint_list) - 1)
        
        path_locations = []
        for i in sample_indices:
            wp = waypoint_list[i]
            loc_id = (wp.road_id, wp.lane_id)
            path_locations.append(loc_id)
            
            if loc_id in used_locations:
                path_overlaps = True
                break
        
        # If no overlap, add this path
        if not path_overlaps:
            selected_paths.append((start_wp, end_wp, distance, waypoint_list))
            
            # Mark all locations in this path as used
            for wp in waypoint_list:
                loc_id = (wp.road_id, wp.lane_id)
                used_locations.add(loc_id)
            
            if len(selected_paths) >= top_n:
                break
    
    print(f"\nSelected {len(selected_paths)} non-overlapping paths:\n")
    
    # for i, (start_wp, end_wp, distance, wps) in enumerate(selected_paths, 1):
    #     print(f"{i}. Distance: {distance:.2f}m, Waypoints: {len(wps)}")
    #     print(f"   Start: Road {start_wp.road_id}, Lane {start_wp.lane_id}, Loc: ({start_wp.transform.location.x:.1f}, {start_wp.transform.location.y:.1f})")
    #     print(f"   End:   Road {end_wp.road_id}, Lane {end_wp.lane_id}, Loc: ({end_wp.transform.location.x:.1f}, {end_wp.transform.location.y:.1f})")
    #     print()
    
    # Convert waypoint lists to formatted routes before returning
    formatted_paths = []
    for start_wp, end_wp, distance, waypoint_list in selected_paths:
        # Convert to (waypoint, RoadOption) format
        formatted_route = [(wp, RoadOption.LANEFOLLOW) for wp in waypoint_list]
        formatted_paths.append((start_wp, end_wp, distance, formatted_route))
    
    return formatted_paths


def visualize_top_paths(world, carla_map, top_paths, lifetime=10.0, step=1.0):
    """
    Visualize the top longest non-overlapping paths.
    
    Args:
        world (carla.World)
        carla_map (carla.Map)
        top_paths (list): List of (start_wp, end_wp, distance, waypoint_list) tuples
        lifetime (float): Seconds to keep drawings
        step (float): Distance between waypoints
    """
    debug = world.debug
    
    for idx, (start_wp, end_wp, distance, waypoint_list) in enumerate(top_paths):
        
        # Draw start marker
        debug.draw_string(
            location=start_wp.transform.location + carla.Location(z=2.0),
            text=f"Start {idx + 1} ({distance:.0f}m)",
            color=carla.Color(0, 255, 0),
            life_time=lifetime
        )
        
        # Draw end marker
        debug.draw_string(
            location=end_wp.transform.location + carla.Location(z=2.0),
            text=f"End {idx + 1}",
            color=carla.Color(255, 0, 0),
            life_time=lifetime
        )
        
        # Draw the path using the stored waypoint list
        for i in range(len(waypoint_list) - 1):
            current_wp = waypoint_list[i][0]
            next_wp = waypoint_list[i + 1][0]
            
            debug.draw_line(
                begin=current_wp.transform.location + carla.Location(z=0.5),
                end=next_wp.transform.location + carla.Location(z=0.5),
                thickness=0.3,
                color=carla.Color(0, 0, 255),
                life_time=lifetime
            )
    
    print(f"Visualized {len(top_paths)} non-overlapping paths")


# Main execution
client = carla.Client('localhost', 2000)
world = client.get_world()
carla_map = world.get_map()

# Find top 20 longest non-junction paths
top_20_paths = find_longest_non_junction_paths(carla_map, top_n=20)

# Visualize them
visualize_top_paths(world, carla_map, top_20_paths, lifetime=30.0)

Building road segment graph...
Found 4210 total road segments

Selected 20 non-overlapping paths:

Visualized 20 non-overlapping paths


In [6]:
import sys

sys.path.append('C:/CARLA_0.9.15/PythonAPI/carla')
from agents.navigation.global_route_planner import GlobalRoutePlanner

sampling_resolution = 1
grp = GlobalRoutePlanner(carla_map, sampling_resolution)

path_idx = 15

route = grp.trace_route(top_20_paths[path_idx][0].transform.location, top_20_paths[path_idx][1].transform.location)

def draw_route(route, world, seconds=5.0):
    for i, (waypoint, _) in enumerate(route):
        world.debug.draw_box(
            carla.BoundingBox(waypoint.transform.location, carla.Vector3D(0.15, 0.15, 0.15)),
            carla.Rotation(),
            thickness=0.5,
            color=carla.Color(r=0, g=0, b=255),
            life_time=seconds
        )

    print(f"Displayed {len(route)} waypoints on the map")
    
# Draw the route
draw_route(route, world, seconds=20.0)

Displayed 85 waypoints on the map


In [21]:
test_route = top_20_paths[0][3]
print(len(test_route))
print(test_route[0])

174
(<carla.libcarla.Waypoint object at 0x00000216668AB820>, <RoadOption.LANEFOLLOW: 4>)


In [22]:
print(len(route))
print(route[0])

174
(<carla.libcarla.Waypoint object at 0x0000021666258F90>, <RoadOption.LANEFOLLOW: 4>)
